In [1]:
import os
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.3.0
      /_/

Using Python version 2.7.6 (default, Mar 22 2014 22:59:56)
SparkContext available as sc, HiveContext available as sqlCtx.


In [2]:
import json

json_data = sc.textFile("/recsys/DO_record_per_line.json")
list_json = json_data.map(lambda x: json.loads(x))
list_json.first()

{u'cat': u'3/business_management|6/economics_finance',
 u'desc': u'This course introduces the basic financial statements used by most businesses, as well as the essential tools used to prepare them. This course will serve as a resource to help business students succeed in their upcoming university-level accounting classes, and as a refresher for upper division accounting students who are struggling to recall elementary concepts essential to more advanced accounting topics. Business owners will also benefit from this class by gaining essential skills necessary to organize and manage information pertinent to operating their business. At the conclusion of the class, students will understand the balance sheet, income statement, and cash flow statement. They will be able to differentiate between cash basis and accrual basis techniques, and know when each is appropriate. They\u2019ll also understand the accounting equation, how to journalize and post transactions, how to adjust and close acc

In [3]:
df_json = sqlCtx.jsonFile("/recsys/DO_record_per_line.json")
df_json.show()

cat                  desc                 id lang name                 provider      
3/business_manage... This course intro... 4  en   Accounting Cycle:... Canvas Network
11/law               This online cours... 5  en   American Counter ... Canvas Network
5/computer_scienc... This course is ta... 6  fr   Arithmétique: en ... Canvas Network
14/social_sciences   We live in a digi... 7  en   Becoming a Dynami... Canvas Network
2/biology_life_sc... This self-paced c... 8  en   Bioethics            Canvas Network
9/humanities|15/m... This game-based c... 9  en   College Foundatio... Canvas Network
14/social_sciences   What’s in your di... 10 en   Digital Literacies I Canvas Network
14/social_sciences   The goal of the D... 11 en   Digital Literacie... Canvas Network
14/social_sciences   Ready to explore ... 12 en   Digital Tools for... Canvas Network
14/social_sciences   This self-paced c... 13 en   Discover Your Val... Canvas Network
12/medicine_health   What is “interpro... 14 en   Enha

In [4]:
import nltk
from nltk.stem.snowball import SnowballStemmer
""""
from nltk.stem.api import StemmerI
from nltk.stem.regexp import RegexpStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.isri import ISRIStemmer
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.rslp import RSLPStemmer
"""
import re
from pymystem3 import Mystem
from bs4 import BeautifulSoup # удобная библиотека для обработки html-тегов, которые есть в текстах

m = Mystem()
#wnl = WordNetLemmatizer()
swEn = nltk.corpus.stopwords.words('english')
swRu = nltk.corpus.stopwords.words('russian')
swEs = nltk.corpus.stopwords.words('spanish')
reWords = re.compile(ur'[\u0400-\u0500a-z\d]+')#\u00C0-\u00FF áéíóúñü

def getWords(text,lang="en",minLen=2):
    text = BeautifulSoup(text).get_text() # удаляем html-теги
    if lang == "ru":
        words = m.lemmatize(text)
        words = [token.lower() for token in words if any([c for c in token.strip() if c.isalpha()])]
    else:
        words = reWords.findall(text.lower())
        words = [token for token in words if any([c for c in token.strip() if c.isalpha()])]
    
    if minLen > 0:
        words = [word for word in words if len(word) >= minLen]
        
    if lang == 'en':
        stopWords = swEn
    elif lang == 'ru':
        stopWords = swRu
    elif lang == 'es':
        stopWords = swEs
    else:
        stopWords = []
    
    if len(stopWords) > 0: 
        words = [word for word in words if word not in stopWords]
        
    if lang == "en":
        #stemmer = PorterStemmer()
        #stemmer = LancasterStemmer()
        stemmer = SnowballStemmer("english")
        words = [stemmer.stem(t) for t in words]
        #words = [wnl.lemmatize(t, pos='v') for t in words]
        #words = [wnl.lemmatize(t) for t in words]
    elif lang == "es":
        stemmer = SnowballStemmer("spanish")
        words = [stemmer.stem(t) for t in words]
       
    return words

def getWords2(text,lang='en',minLen=2):
    text = BeautifulSoup(text).get_text()
    words = reWords.findall(text.lower())
    words = [token for token in words if any([c for c in token.strip() if c.isalpha()])]
    if minLen > 0:
        words = [word for word in words if len(word) >= minLen]
        
    if lang == 'en':
        stopWords = swEn
    elif lang == 'ru':
        stopWords = swRu
    elif lang == 'es':
        stopWords = swEs
    else:
        stopWords = []    
    if len(stopWords) > 0: 
        words = [word for word in words if word not in stopWords]
        
    return words

def getWords3(text):
    return reWords.findall(text.lower())

from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(ur'[\u0400-\u0500a-z\d]+')
tokenizerBr = sc.broadcast( tokenizer )

def getWords4(text):
    return tokenizerBr.value.tokenize(text.lower())

In [ ]:
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.feature import IDF
from pyspark.mllib.feature import Normalizer

def calc_cosine_similarity(a,b):
    return a.dot(b)

def fchosen_doc(idnum,df_rec=None):
    if df_rec == None:
        df_rec = df_json[df_json.id == idnum].first()
    return normalizer_l2.transform(idf_fit.transform(tf.transform(getWords3(df_rec[1]))))

normalizer_l2 = Normalizer(p=2)
tf, idf = HashingTF(10000), IDF()

tokens = df_json.map(lambda x: getWords3(x.desc))
doc_tf = tf.transform(tokens)

idf_fit = idf.fit(doc_tf)
tfidf = idf_fit.transform(doc_tf)
normalized_tf_idf_vectors = normalizer_l2.transform(tfidf)

In [ ]:
ids = [14398, 24543, 12873, 13449, 8200, 1133]
top11 = list()

def getCat(text):
    if text > '':
        ok = text.split('|')
        ok = [int(x.split('/')[0]) for x in ok]
    else:
        ok = []
    return set(ok)
     
for idnum in ids:
    df_rec = df_json[df_json.id == idnum].first()
    lang = df_rec[3]
    #cats = getCat(df_rec[0])
    #ncats = len(cats)

    chosen_doc = fchosen_doc(idnum,df_rec)
    #chosen_doc_br = sc.broadcast(chosen_doc)
    similarities = normalized_tf_idf_vectors.map(lambda x: calc_cosine_similarity(x, chosen_doc))

    listid = df_json.map(lambda x: x.id).collect()
    listok = similarities.collect()
    ok = sorted(zip(listid,listok),key = lambda x: -x[1])[:11]

    #ok = df_json.map(lambda x: x.id).zip(similarities).sortBy(lambda x: -x[1]).take(11)

    top11.append((idnum,lang,ok))
    print idnum

In [ ]:
top11

In [ ]:
import json

ok = dict()
for val in top11:
    s = str(val[0])
    ok[s] = map(lambda x: x[0],filter(lambda x: x[0] != val[0],val[2]))
out = json.dumps(ok, sort_keys=False)
print out
f = open(r"lab10.json","w")
f.write(out)
f.close()

In [15]:
for x in top11:
    print x[0],x[1]
    for y in x[2]:
        print '  ',y[0],',',y[1],':',df_json[df_json.id == y[0]].collect()[0][4]

14398 en
   14398 , 1.0 : CSS with LESS and Sass
   23666 , 0.253505777821 : Getting Started with LESS - Beginner Crash Course
   11263 , 0.246166898516 : Less CSS Tutorial
   25066 , 0.242909423088 : Rapid LESS - Udemy
   14651 , 0.234694248712 : FileMaker Pro 10 Essential Training
   13348 , 0.22774698939 : Learning Sass - CSS Just Got Exciting. by Infinite Skills
   25782 , 0.220259942493 : Write Clean CSS using SASS - Udemy
   17665 , 0.213201814327 : Writing CSS with {LESS}
   25071 , 0.21032699738 : Rapid SASS - Udemy
   18368 , 0.209791608146 : Building High-Performance Teams
   13665 , 0.207571901239 : The Next Step with Sass and Compass by Lisa Catalano
24543 en
   24543 , 1.0 : Love Your Life - Why Settle? Be Happy - Dream Builder Course
   6174 , 0.460623659535 : How to Find True Love: Making Way for Love!
   21009 , 0.442319945906 : You Part Two: Discover your Life Purpose - Udemy
   11616 , 0.438798066627 : Solve Relationship Problems in 12 Easy Steps by Sarah McCrum
   10

In [ ]:
print df_json[df_json.id == 12802].collect()[0][4]

In [16]:
print df_json[df_json.id == 12658].collect()[0][4]

Aprende a tocar el piano desde cero by Virtuosso Producciones
